In [1]:
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_columns',None)

In [2]:


# Load your JSON data from the file
with open('weather_data_extended.json', 'r') as file:
    data = json.load(file)

# Assuming 'data' is a list of records, where each record corresponds to a city's weather data
flattened_data = []

for record in data:
    # Flatten the 'list' key in your JSON (which contains the weather data)
    try:
        weather_data = pd.json_normalize(record,'list')

        del weather_data["weather"]

        weather_data["weather"] = pd.json_normalize(record['list'],"weather")["main"]
    
        
        # Flatten the 'city' data and join with weather data
        city_data = pd.json_normalize(record['city'])[["name","coord.lat","coord.lon"]]
        city_data = pd.concat([city_data]*len(weather_data), ignore_index=True)
        
        # print(pd.json_normalize(record['city']))

        # Combine city data with weather data
        combined_data = pd.concat([city_data, weather_data], axis=1)
        flattened_data.append(combined_data)

    except:
        print("Null record encountered. Proceeding anyways")

# Concatenate all the flattened data
df = pd.concat(flattened_data, ignore_index=True)

# Display the DataFrame
df.head()


Null record encountered. Proceeding anyways
Null record encountered. Proceeding anyways


,name,coord.lat,coord.lon,dt,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,main.sea_level,main.grnd_level,main.humidity,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod,rain.3h,weather,snow.3h
0,New York,40.7143,-74.006,1701432000,10000,0.0,2023-12-01 12:00:00,277.67,275.26,277.67,278.17,1022,1022,1021,74,-0.5,11,2.77,219,6.10,n,NaN,Clouds,NaN
1,New York,40.7143,-74.006,1701442800,10000,0.0,2023-12-01 15:00:00,279.41,276.98,279.41,280.41,1022,1022,1021,67,-1.0,48,3.28,211,5.88,d,NaN,Clouds,NaN
2,New York,40.7143,-74.006,1701453600,10000,0.0,2023-12-01 18:00:00,281.65,280.23,281.65,281.65,1020,1020,1020,62,0.0,86,2.45,220,7.06,d,NaN,Clouds,NaN
3,New York,40.7143,-74.006,1701464400,10000,0.1,2023-12-01 21:00:00,281.81,279.69,281.81,281.81,1019,1019,1019,67,0.0,100,3.64,179,9.98,d,NaN,Clouds,NaN
4,New York,40.7143,-74.006,1701475200,7469,1.0,2023-12-02 00:00:00,280.51,278.31,280.51,280.51,1019,1019,1019,91,0.0,100,3.28,202,10.59,n,3.9,Rain,NaN


In [3]:
test_data = df[(df["dt_txt"]>'2023-12-04 00:00:00')& (df["dt_txt"]<'2023-12-05 00:00:00')].reset_index(drop=True)

In [4]:
test_data.to_csv("test_data.csv",index=False)